In [12]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from scipy.signal import butter, filtfilt
from scipy.io import loadmat
import wfdb
from wfdb import rdrecord

In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
# from train_ecg_vae_reutilice import ECGDataset, ECG_VAE, loss_fn

In [15]:
# PTB_DIR = 'data/ptb-xl'
# CH_DIR  = 'data/ChapmanShaoxing'
# # Hyperparams
# batch_size = 16
# epochs = 16
# lr = 1e-3
# z_dim = 16
# seq_len = 5000

# # Dataset & Loader
# ds = ECGDataset(PTB_DIR, CH_DIR, sample_length=seq_len)
# loader = DataLoader(ds, batch_size=batch_size, shuffle=True, num_workers=4)

# # Model, optim
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# vae = ECG_VAE(z_dim=z_dim, seq_len=seq_len).to(device)
# opt = optim.Adam(vae.parameters(), lr=lr)

# # Train
# for ep in range(1, epochs+1):
#     vae.train()
#     tot_loss = 0
#     for batch in loader:
#         x = batch.to(device)
#         x_hat, mu, logv = vae(x)
#         loss, recon, kld = loss_fn(x, x_hat, mu, logv)
#         opt.zero_grad(); loss.backward(); opt.step()
#         tot_loss += loss.item()
#     print(f"Epoch {ep}/{epochs} - Loss: {tot_loss/len(loader):.4f} (Recon {recon:.4f}, KLD {kld:.4f})")
# # Guardar
# torch.save(vae.state_dict(), 'ecg_vae.pth')
# print("Modelo guardado en ecg_vae.pth")


In [16]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from scipy.signal import butter, filtfilt
from scipy.io import loadmat
import wfdb
from wfdb import rdrecord

In [17]:
# from VAE import ECGDataset, ECG_VAE, loss_fn

In [18]:
# PTB_DIR='data/ptb-xl'; CH_DIR='data/ChapmanShaoxing'
# batch=16; epochs=15; lr=1e-3; z_dim=16; seq_len=5000
# # dataset completo
# ds=ECGDataset(PTB_DIR,CH_DIR,seq_len)
# # split: 80% train, 10% val, 10% test
# n=len(ds); n_test=int(0.1*n); n_val=int(0.1*n); n_train=n-n_val-n_test
# ds_train, ds_val, ds_test = random_split(ds, [n_train,n_val,n_test])
# loader_tr=DataLoader(ds_train,batch,shuffle=True,num_workers=4)
# loader_val=DataLoader(ds_val,batch,shuffle=False,num_workers=2)

# device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# vae=ECG_VAE(z_dim=z_dim, seq_len=seq_len).to(device)
# opt=optim.Adam(vae.parameters(), lr=lr)

# best_val=float('inf')
# for ep in range(1,epochs+1):
#     vae.train(); train_loss=0
#     for xb in loader_tr:
#         xb=xb.to(device)
#         xh, mu, lv=vae(xb)
#         loss,_,_ = loss_fn(xb,xh,mu,lv)
#         opt.zero_grad(); loss.backward();
#         nn.utils.clip_grad_norm_(vae.parameters(),1.0)
#         opt.step(); train_loss+=loss.item()
#     train_loss/=len(loader_tr)
#     # validación
#     vae.eval(); val_loss=0
#     with torch.no_grad():
#         for xb in loader_val:
#             xb=xb.to(device)
#             xh,mu,lv=vae(xb)
#             loss,_,_=loss_fn(xb,xh,mu,lv)
#             val_loss+=loss.item()
#     val_loss/=len(loader_val)
#     print(f"Epoch {ep}/{epochs} - Train L: {train_loss:.4f}  Val L: {val_loss:.4f}")
#     if val_loss<best_val:
#         best_val=val_loss
#         torch.save(vae.state_dict(),'ecg_vae_best.pth')
# print("Entrenamiento completo. Mejor modelo en ecg_vae_best.pth")
# # test: MAE sobre ds_test
# from torch.utils.data import DataLoader
# loader_te=DataLoader(ds_test,batch,shuffle=False)
# tot_mae=[]
# vae.load_state_dict(torch.load('ecg_vae_best.pth',map_location=device))
# vae.eval()
# with torch.no_grad():
#     for xb in loader_te:
#         xb=xb.to(device)
#         xh,_,_=vae(xb)
#         mae=torch.abs(xh-xb).mean(dim=(1,2)).cpu().numpy()
#         tot_mae.extend(mae.tolist())
# print(f"MAE promedio normales test: {np.mean(tot_mae):.4f}")
# # ds_test contiene sólo normales. Anómalos se evalúan en script aparte.

In [19]:
from VAEnew import ECGDataset, ECG_VAE, loss_fn

In [20]:
PTB_DIR, CH_DIR = 'data/ptb-xl', 'data/ChapmanShaoxing'
batch_size, seq_len, z_dim = 16, 5000, 16
epochs, lr, beta = 20, 1e-4, 5.0

# Dataset y splits
ds = ECGDataset(PTB_DIR, CH_DIR, sample_length=seq_len)
n = len(ds)
n_test = int(0.1 * n)
n_val = int(0.1 * n)
n_train = n - n_val - n_test
ds_tr, ds_val, ds_test = random_split(ds, [n_train, n_val, n_test])
loader_tr = DataLoader(ds_tr, batch_size, shuffle=True, num_workers=4)
loader_val= DataLoader(ds_val, batch_size, shuffle=False, num_workers=2)
loader_te = DataLoader(ds_test, batch_size, shuffle=False, num_workers=2)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
vae = ECG_VAE(in_ch=12, z_dim=z_dim, seq_len=seq_len).to(device)
optimizer = optim.Adam(vae.parameters(), lr=lr)

best_val_loss = float('inf')
for epoch in range(1, epochs+1):
    # Entrenamiento
    vae.train()
    train_loss = 0.0
    for xb in loader_tr:
        xb = xb.to(device)
        xh, mu, logv = vae(xb)
        loss, recon, kld = loss_fn(xb, xh, mu, logv, beta)
        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(vae.parameters(), 1.0)
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(loader_tr)

    # Validación
    vae.eval()
    val_loss, recs, klds = 0.0, [], []
    with torch.no_grad():
        for xb in loader_val:
            xb = xb.to(device)
            xh, mu, logv = vae(xb)
            loss, recon, kld = loss_fn(xb, xh, mu, logv, beta)
            val_loss += loss.item()
            recs.append(recon.item())
            klds.append(kld.item())
    val_loss /= len(loader_val)
    # Guardar mejor modelo
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(vae.state_dict(), 'ecg_vae_best.pth')
        np.save('val_scores.npy', np.stack([recs, klds], axis=1))
        mus = []
        with torch.no_grad():
            for xb in loader_val:
                xb = xb.to(device)
                _, mu_batch, _ = vae(xb)
                mus.append(mu_batch.cpu().numpy())
        np.save('val_mu.npy', np.concatenate(mus, axis=0))

    print(f"Epoch {epoch}/{epochs} - Train Loss: {train_loss:.4f}  Val Loss: {val_loss:.4f}")

print("Entrenamiento completo. Mejor modelo guardado en ecg_vae_best.pth")

# Test finales (normales)
vae.load_state_dict(torch.load('ecg_vae_best.pth', map_location=device))
te_recs, te_klds = [], []
vae.eval()
with torch.no_grad():
    for xb in loader_te:
        xb = xb.to(device)
        xh, mu, logv = vae(xb)
        mse_vals = nn.functional.mse_loss(xh, xb, reduction='none').mean(dim=(1,2))
        kld_vals = (-0.5 * torch.sum(1 + logv - mu.pow(2) - logv.exp(), dim=1))
        te_recs.extend(mse_vals.cpu().numpy())
        te_klds.extend(kld_vals.cpu().numpy())
np.save('test_scores.npy', np.stack([te_recs, te_klds], axis=1))
mus_test = []
with torch.no_grad():
    for xb in loader_te:
        xb = xb.to(device)
        _, mu_batch, _ = vae(xb)
        mus_test.append(mu_batch.cpu().numpy())
np.save('test_mu.npy', np.concatenate(mus_test, axis=0))
print(f"Test Normales - MSE mean: {np.mean(te_recs):.4f}, KLD mean: {np.mean(te_klds):.4f}")


Epoch 1/20 - Train Loss: 1.1233  Val Loss: 1.0313
Epoch 2/20 - Train Loss: 1.0081  Val Loss: 1.0025
Epoch 3/20 - Train Loss: 1.0011  Val Loss: 1.0006
Epoch 4/20 - Train Loss: 1.0005  Val Loss: 1.0002
Epoch 5/20 - Train Loss: 1.0002  Val Loss: 1.0001
Epoch 6/20 - Train Loss: 1.0001  Val Loss: 1.0001
Epoch 7/20 - Train Loss: 1.0001  Val Loss: 1.0001
Epoch 8/20 - Train Loss: 1.0000  Val Loss: 1.0000
Epoch 9/20 - Train Loss: 1.0000  Val Loss: 1.0000
Epoch 10/20 - Train Loss: 1.0000  Val Loss: 1.0000
Epoch 11/20 - Train Loss: 1.0000  Val Loss: 1.0000
Epoch 12/20 - Train Loss: 1.0000  Val Loss: 1.0002
Epoch 13/20 - Train Loss: 1.0000  Val Loss: 1.0000
Epoch 14/20 - Train Loss: 1.0000  Val Loss: 1.0000
Epoch 15/20 - Train Loss: 1.0000  Val Loss: 1.0000
Epoch 16/20 - Train Loss: 1.0000  Val Loss: 1.0000
Epoch 17/20 - Train Loss: 1.0000  Val Loss: 1.0000
Epoch 18/20 - Train Loss: 1.0000  Val Loss: 1.0000
Epoch 19/20 - Train Loss: 1.0000  Val Loss: 1.0000
Epoch 20/20 - Train Loss: 1.0000  Val Lo